**DEMO CODE - DO NOT USE IN PRODUCTION**

In [ ]:
###########################################################################
#
#  Copyright 2023 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available
# on an “as is,” “as available,” and “with all faults” basis, solely for
# illustrative purposes, and without warranty or representation of any kind.
# This solution is experimental, unsupported and provided solely for your
# convenience. Your use of it is subject to your agreements with Google, as
# applicable, and may constitute a beta feature as defined under those
# agreements.  To the extent that you make any data available to Google in
# connection with your use of the solution, you represent and warrant that you
# have all necessary and appropriate rights, consents and permissions to permit
# Google to use and process that data.  By using any portion of this solution,
# you acknowledge, assume and accept all risks, known and unknown, associated
# with its usage, including with respect to your deployment of any portion of
# this solution in your systems, or usage in connection with your business,
# if at all.
###########################################################################

# Demo Setup

## Import libraries

In [ ]:
!pip install -q icecream

import pandas as pd
from google.colab import auth
from google.cloud import bigquery
from icecream import ic

ic("Imports completed")

## Connect to data source

In [ ]:
# Load libraries and authenticate colab for BQ

auth.authenticate_user()
bigquery.USE_LEGACY_SQL = False

ic("User authenticated")

## Set project parameters

In [ ]:
#The identifier of your Google Cloud project
project_id = "db-platform-sol" #@param

# Variable related to your GA dataset and tables
ga_project_id = "bigquery-public-data" #@param
ga_dataset = "ga4_obfuscated_sample_ecommerce" #@param
ga_data = "events_*" #@param





In [ ]:

client = bigquery.Client(location="US", project=project_id)
print(f"BQ client created using project ID: {client.project}")

# DEMO 1: Automated Query visualization

In [ ]:
query = f"""
    SELECT user_pseudo_id,
      event_date as first_transaction_date,
      ecommerce.purchase_revenue as purchase_rev,
      geo.continent as geo,
      device.category as device,
      device.operating_system as device_os,
      traffic_source.medium as source
    FROM `{ga_project_id}.{ga_dataset}.{ga_data}`
    WHERE ecommerce.purchase_revenue >= 1
"""


query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()
df



# DEMO 2: Propensity to buy model

## Model training

In [ ]:
model_dataset = "GA4_Training_Day" #@param
model_name = "ga4_purchase_propensity_model_Aug20" #@param

# First-party data used to augment the GA4 data
crm_table = "crm_synthetic_customer_info" #@param
crm_transactions = "uploaded_synthetic_offline_transactions" #@param




In [ ]:
training_query = f"""

    #Create GA4 Predictive Model

CREATE OR REPLACE MODEL `{project_id}.{model_dataset}.{model_name}`
    OPTIONS
        (model_type='LOGISTIC_REG',
        calculate_p_values=TRUE,
        CATEGORY_ENCODING_METHOD='DUMMY_ENCODING',
        input_label_cols = ['future_purchase']) AS


#Features

#CRM Customer Data
WITH customer_info AS (
SELECT
    Customer_Id AS customer_id,
    Age AS crm_age,
    Super_Saver_Club AS crm_super_saver_club
FROM
    `{project_id}.{model_dataset}.{crm_table}`
),

#GA4 User Activity Data
historical_site_visits AS (
SELECT
    user_pseudo_id AS customer_id,
    PARSE_DATE('%Y%m%d',event_date) AS event_date,
    SUM(CASE WHEN event_name = 'page_view' THEN 1 ELSE 0 END) AS ga_page_view_count,
    SUM(CASE WHEN event_name = 'view_item' THEN 1 ELSE 0 END) AS ga_view_item_count,
    MAX(CASE WHEN event_name = 'first_visit' THEN 1 ELSE 0 END) AS ga_first_visit_dummy,
    MAX(CASE WHEN event_name = 'view_promotion' THEN 1 ELSE 0 END) AS ga_view_promotion_dummy,
    MAX(CASE WHEN event_name = 'add_to_cart' THEN 1 ELSE 0 END) AS ga_add_to_cart_dummy,
    MAX(CASE WHEN event_name = 'begin_checkout' THEN 1 ELSE 0 END) AS ga_begin_checkout_dummy,
    MAX(CASE WHEN event_name IN ('purchase', 'in_app_purchase') THEN 1 ELSE 0 END) AS ga_online_purchase_dummy
FROM
    `{ga_project_id}.{ga_dataset}.{ga_data}`
WHERE
    _TABLE_SUFFIX BETWEEN '20201101' AND '20201231'
GROUP BY
    1,2
),

#CRM transaction data - target variable to predict
purchases AS (
(SELECT
    user_pseudo_id AS customer_id,
    PARSE_DATE('%Y%m%d',event_date) AS transaction_date,
    MAX(CASE WHEN event_name IN ('purchase', 'in_app_purchase') THEN 1 ELSE 0 END) AS future_purchase
FROM
    `{ga_project_id}.{ga_dataset}.{ga_data}`
WHERE
    _TABLE_SUFFIX BETWEEN '20201101' AND '20210107'
GROUP BY
    customer_id,
    transaction_date
)
UNION ALL
(SELECT
    customer_id,
    transaction_date,
    MAX(CASE WHEN transaction_type = 'purchase' AND offline_revenue > 0 THEN 1 ELSE 0 END) AS future_purchase
FROM
    `{project_id}.{model_dataset}.{crm_transactions}`
WHERE
    transaction_date BETWEEN '2020-11-01' AND '2021-01-07'
GROUP BY
    customer_id,
    transaction_date
)
)

#Feature Table
SELECT
    customer_info.crm_age,
    customer_info.crm_super_saver_club,
    SUM(COALESCE(historical_site_visits.ga_page_view_count,0)) AS ga_page_view_count,
    SUM(COALESCE(historical_site_visits.ga_view_item_count,0)) AS ga_view_item_count,
    MIN(COALESCE(historical_site_visits.ga_first_visit_dummy,0)) AS ga_first_visit_dummy,
    MAX(COALESCE(historical_site_visits.ga_view_promotion_dummy,0)) AS ga_view_promotion_dummy,
    MAX(COALESCE(historical_site_visits.ga_add_to_cart_dummy,0)) AS ga_add_to_cart_dummy,
    MAX(COALESCE(historical_site_visits.ga_begin_checkout_dummy,0)) AS ga_begin_checkout_dummy,
    MAX(COALESCE(historical_site_visits.ga_online_purchase_dummy,0)) AS ga_online_purchase_dummy,
    MAX(COALESCE(purchases.future_purchase,0)) AS future_purchase
FROM
    customer_info
LEFT JOIN
    historical_site_visits
ON
    customer_info.Customer_Id = historical_site_visits.customer_id
LEFT JOIN
    purchases
ON
    historical_site_visits.customer_id = purchases.customer_id
    AND DATE_DIFF(purchases.transaction_date, historical_site_visits.event_date, DAY) BETWEEN 1 AND 7
GROUP BY
    customer_info.customer_id,
    customer_info.crm_age,
    customer_info.crm_super_saver_club
"""

ic("Training query set. Ready to start training")

In [ ]:
training_job = client.query(
    training_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query


training_job.result()  # Waits for the query to finish
ic("Model training completed")

## Predictions using trained model

In [ ]:
model_for_predictions = "ga4_purchase_propensity_model_Aug20" #@param

In [ ]:
prediction_query = f"""


#Create sample predictions using model

SELECT
    customer_id,
    predicted_future_purchase_probs.prob,
    NTILE(10) OVER(ORDER BY predicted_future_purchase_probs.prob DESC) AS decile,
    future_purchase,
    channel
FROM
    ML.PREDICT(MODEL `{project_id}.{model_dataset}.{model_for_predictions}`,
(WITH customer_info AS (
SELECT
    Customer_Id AS customer_id,
    Age AS crm_age,
    Super_Saver_Club AS crm_super_saver_club
FROM
    `db-platform-sol.GA4_Training_Day.crm_synthetic_customer_info`
),

historical_site_visits AS (
SELECT
    user_pseudo_id AS customer_id,
    PARSE_DATE('%Y%m%d',event_date) AS event_date,
    SUM(CASE WHEN event_name = 'page_view' THEN 1 ELSE 0 END) AS ga_page_view_count,
    SUM(CASE WHEN event_name = 'view_item' THEN 1 ELSE 0 END) AS ga_view_item_count,
    MAX(CASE WHEN event_name = 'first_visit' THEN 1 ELSE 0 END) AS ga_first_visit_dummy,
    MAX(CASE WHEN event_name = 'view_promotion' THEN 1 ELSE 0 END) AS ga_view_promotion_dummy,
    MAX(CASE WHEN event_name = 'add_to_cart' THEN 1 ELSE 0 END) AS ga_add_to_cart_dummy,
    MAX(CASE WHEN event_name = 'begin_checkout' THEN 1 ELSE 0 END) AS ga_begin_checkout_dummy,
    MAX(CASE WHEN event_name IN ('purchase', 'in_app_purchase') THEN 1 ELSE 0 END) AS ga_online_purchase_dummy
FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20210108' AND '20210124'
GROUP BY
    1,2
),
purchases AS (
(SELECT
    user_pseudo_id AS customer_id,
    PARSE_DATE('%Y%m%d',event_date) AS transaction_date,
    MAX(CASE WHEN event_name IN ('purchase', 'in_app_purchase') THEN 1 ELSE 0 END) AS future_purchase,
    CASE WHEN event_name IN ('purchase', 'in_app_purchase') THEN 'online' END AS channel
FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20210108' AND '20210131'
GROUP BY
    customer_id,
    transaction_date,
    channel
)
UNION ALL
(SELECT
    customer_id,
    transaction_date,
    MAX(CASE WHEN transaction_type = 'purchase' AND offline_revenue > 0 THEN 1 ELSE 0 END) AS future_purchase,
    'in store' AS channel
FROM
    `db-platform-sol.GA4_Training_Day.uploaded_synthetic_offline_transactions`
WHERE
    transaction_date BETWEEN '2021-01-08' AND '2021-01-31'
GROUP BY
    customer_id,
    transaction_date
)
)

SELECT
    customer_info.customer_id,
    customer_info.crm_age,
    customer_info.crm_super_saver_club,
    SUM(COALESCE(historical_site_visits.ga_page_view_count,0)) AS ga_page_view_count,
    SUM(COALESCE(historical_site_visits.ga_view_item_count,0)) AS ga_view_item_count,
    MIN(COALESCE(historical_site_visits.ga_first_visit_dummy,0)) AS ga_first_visit_dummy,
    MAX(COALESCE(historical_site_visits.ga_view_promotion_dummy,0)) AS ga_view_promotion_dummy,
    MAX(COALESCE(historical_site_visits.ga_add_to_cart_dummy,0)) AS ga_add_to_cart_dummy,
    MAX(COALESCE(historical_site_visits.ga_begin_checkout_dummy,0)) AS ga_begin_checkout_dummy,
    MAX(COALESCE(historical_site_visits.ga_online_purchase_dummy,0)) AS ga_online_purchase_dummy,
    MAX(COALESCE(channel,'')) AS channel,
    MAX(COALESCE(purchases.future_purchase,0)) AS future_purchase
FROM
    customer_info
LEFT JOIN
    historical_site_visits
ON
    customer_info.Customer_Id = historical_site_visits.customer_id
LEFT JOIN
    purchases
ON
    historical_site_visits.customer_id = purchases.customer_id
    AND DATE_DIFF(purchases.transaction_date, historical_site_visits.event_date, DAY) BETWEEN 1 AND 7
GROUP BY
    customer_info.customer_id,
    customer_info.crm_age,
    customer_info.crm_super_saver_club
)
),
UNNEST(predicted_future_purchase_probs) predicted_future_purchase_probs
WHERE
    predicted_future_purchase_probs.label = 1

"""

ic("Prediction query set. Ready to start prediction")



In [ ]:
prediction_job = client.query(
    prediction_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = prediction_job.to_dataframe()
df.info()

In [ ]:
df

# DEMO 3: Generative AI with BigQuery

In [ ]:
project_id = "db-platform-sol" #@param
genai_model = "remote_llm_ga4" #@param


In [ ]:
client = bigquery.Client(location="US", project=project_id)

## Create LLM remote model

In [ ]:
# Only run once to create the reference to the remote LLM model

query = f"""
  CREATE OR REPLACE MODEL  {model_dataset}.{genai_model}
    REMOTE WITH CONNECTION `us.connection-vertex-llm`
    OPTIONS (remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1')

"""
# ic(f"Ready to connect to GEN AI model" {genai_model})
ic(" Gen AI model ready to be created")
ic(genai_model)


In [ ]:


query_job = client.query(
    query,
    location="US",
)  # API request - starts the query


query_job.result()  # Waits for the query to finish
ic("Connection to Google Cloud Generative AI model created")

## Gen AI for GA4 promotion campaigns

In [ ]:
# Use Generative AI to create a campaign decription based on information stored in BigQuery
# The prompt uses GA4 data: items.promotion_name

gen_ai_query = f"""

  SELECT *
    FROM
      ML.GENERATE_TEXT(
        MODEL {model_dataset}.{genai_model},
        (
        SELECT CONCAT('Create a promotion campaign description using the following promotion name: ',promotion_name) AS prompt
        FROM GA4_Training_Day.promotion_campaigns
        ),
        STRUCT(
          0.2 AS temperature, 600 AS max_output_tokens, 0.2 AS top_p, 15 AS top_k))
"""


ic("Generative AI query set. Ready to start generation")

In [ ]:
gen_ai_job = client.query(
    gen_ai_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = gen_ai_job.to_dataframe()
df





## Gen AI for GA4 item names

In [ ]:
product_table = "db-platform-sol.GA4_Training_Day.product_names"



temp = 0.2 #@param
max_tokens = 650 #@param
top_p = 0.2 #@param
top_k = 15 #@param

# e.g
prompt = "Create a product description for a product based on its name" #@param

In [ ]:
# Use Generative AI to create a campaign decription based on information stored in BigQuery
# The prompt uses GA4 data: items.promotion_name

gen_ai_query = f"""

  SELECT *
    FROM
      ML.GENERATE_TEXT(
      MODEL {model_dataset}.{genai_model},
      (
      SELECT CONCAT('{prompt}',' Product name: ', item_name) AS prompt
      FROM `{product_table}`
      LIMIT 5
      ),
      STRUCT(
        {temp} AS temperature, {max_tokens} AS max_output_tokens, {top_p} AS top_p, {top_k} AS top_k));

"""


ic("Generative AI query set. Ready to start generation")

In [ ]:
gen_ai_job = client.query(
    gen_ai_query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = gen_ai_job.to_dataframe()
df